# [Day 7: No Space Left On Device](https://adventofcode.com/2022/day/7)

## Part 1

In [1]:
import collections
import dataclasses
import pathlib
import re
from typing import Optional

In [2]:
test_terminal_output = """
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
"""

In [3]:
RE_LINE = re.compile("""
^\s*
(
  \$\s*
  ((?P<cd>cd\s+(?P<cd_name>\S+))
  |
  (?P<ls>ls)
)
|
(?P<dir>dir\s+(?P<dir_name>.*))
|
(?P<file>(?P<file_size>\d+)\s+(?P<file_name>.*)\s*)
)
\s*$
""", re.IGNORECASE | re.VERBOSE)

@dataclasses.dataclass
class File:
    name: str
    size: int
    
@dataclasses.dataclass
class Directory:
    name: str
    directories: dict[str, "Directory"] = dataclasses.field(default_factory=dict)
    files: dict[str, File] = dataclasses.field(default_factory=dict)
    size: int = 0
    parent: Optional["Directory"] = None

    def add_directory(self, name: str) -> None:
        new_directory = Directory(name=name)
        if name != "/":
            new_directory.parent = self
        self.directories[name] = new_directory

    def add_file(self, name: str, size: int) -> None:
        self.files[name] = File(name=name, size=size)
        self.size += size
        # walk up tree and update sizes
        parent = self
        while (parent := parent.parent) is not None:
            parent.size += size
    
    def cd(self, name: str) -> "Directory":
        # cd /
        if name == "/":
            # walk up tree and return the root
            parent, root = self, self
            while (parent := parent.parent) is not None:
                root = parent
            return root
        # cd ..
        if name == "..":
            if self.parent is None:
                return self  # already at root
            else:
                return self.parent
        # cd name
        return self.directories[name]
        
@dataclasses.dataclass
class Filesystem:
    root: None | Directory = None
    
    def __post_init__(self) -> None:
        self.root = Directory("/")
    
    def replay_terminal_output(self, term_out: str) -> None:
        current_dir = self.root
        for line in term_out.splitlines():
            if mt_line := RE_LINE.match(line):
                if mt_line["cd"]:
                    current_dir = current_dir.cd(mt_line["cd_name"])
                elif mt_line["dir"]:
                    current_dir.add_directory(mt_line["dir_name"])
                elif mt_line["file"]:
                    current_dir.add_file(mt_line["file_name"], int(mt_line["file_size"]))

In [4]:
def total_size_with_max(filesystem: Filesystem, max_size: int) -> int:
    total_size = 0
    todo = collections.deque([filesystem.root])
    while todo:
        directory = todo.pop()
        todo.extend(directory.directories.values())
        size = directory.size
        if size <= max_size:
            total_size += size
    return total_size

In [5]:
%%time
test_filesystem = Filesystem()
test_filesystem.replay_terminal_output(test_terminal_output)

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
%%time
(
    test_filesystem.root.directories["a"].directories["e"].size == 584,
    test_filesystem.root.directories["a"].size == 94853,
    test_filesystem.root.directories["d"].size == 24933642,
    test_filesystem.root.size == 48381165,
    total_size_with_max(test_filesystem, 100000) == 95437,
)

CPU times: total: 0 ns
Wall time: 0 ns


(True, True, True, True, True)

In [7]:
terminal_output = pathlib.Path("../data/day07.txt").read_text().strip()
terminal_output.splitlines()[-1] == "25218 dtdt"

True

In [8]:
%%time
filesystem = Filesystem()
filesystem.replay_terminal_output(terminal_output)

CPU times: total: 0 ns
Wall time: 1 ms


In [9]:
%%time
print(f"Answer part 1: {total_size_with_max(filesystem, 100000)}\n")

Answer part 1: 1297683

CPU times: total: 0 ns
Wall time: 0 ns


## Part 2

In [10]:
def find_directory_to_delete(filesystem: Filesystem, required_space: int):
    candidates = []
    todo = collections.deque([filesystem.root])
    while todo:
        directory = todo.pop()
        todo.extend(directory.directories.values())
        size = directory.size
        if size >= required_space:
            candidates.append(size)
    return sorted(candidates)[0]

def get_directory_to_delete(filesystem: Filesystem, needed_space: int, filesystem_size: int = 70000000):
    current_free_space = filesystem_size - filesystem.root.size
    space_to_free = needed_space - current_free_space
    return find_directory_to_delete(filesystem, space_to_free)

In [11]:
%%time
get_directory_to_delete(test_filesystem, 30000000) == 24933642

CPU times: total: 0 ns
Wall time: 0 ns


True

In [12]:
%%time
print(f"Answer part 2: {get_directory_to_delete(filesystem, 30000000)}\n")

Answer part 2: 5756764

CPU times: total: 0 ns
Wall time: 0 ns
